# Final Supervised Learning on Binary IDC Breast Cancer Classification Task 

In [1]:
import torch, gc
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from datetime import datetime
import pandas as pd
import random 
from torchvision.datasets import ImageFolder
from torchvision.models import resnet
import re
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
from skimage.io import imread, imsave
import skimage
from PIL import ImageFile
from PIL import Image
import argparse
from functools import partial

gc.collect()
torch.cuda.empty_cache()

### Check GPU Info ### 

In [2]:
#show gpu info
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
torch.cuda.current_device()
device = 'cuda'

gpu_info = !nvidia-smi -i 0
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

True
1
Tesla V100-SXM2-16GB
Tue Apr 26 03:39:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:18:00.0 Off |                    0 |
| N/A   46C    P0    45W / 300W |      3MiB / 16160MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

### Define all Argument ###

In [19]:
parser = argparse.ArgumentParser(description='Final Training Classifier on COVIDCT')

parser.add_argument('-a', '--arch', default='resnet18')
parser.add_argument('-bs','--batch-size', default=64, type=int, metavar='N', help='mini-batch size')
parser.add_argument('--epochs', default=1000, type=int, metavar='N', help='number of total epochs to run')
parser.add_argument('--results-dir', default='', type=str, metavar='PATH', help='path to cache (default: none)')
parser.add_argument('--resume',default=False,help='if resume training ')
parser.add_argument('--start-epoch',default = 1, type=int)
parser.add_argument('--total-epoch',default = 100, type=int)
parser.add_argument('--model-load',default = '',help='pretrained model file path')
args = parser.parse_args('')

args.results_dir = './IDC-cache/cache-' + 'train-Cifar-randomini'
#args.model_load = './IDC-cache/cache-moco-Cifar/model_last.pth'
print(args)

Namespace(arch='resnet18', batch_size=64, epochs=1000, model_load='', results_dir='./IDC-cache/cache-train-Cifar-randomini', resume=False, start_epoch=1, total_epoch=100)


### Evaluation Metric ###

In [5]:
#define metric function 

def metric(predlist,tarlist,scorelist):
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    TP = ((predlist == 1) & (tarlist == 1)).sum()
    TN = ((predlist == 0) & (tarlist == 0)).sum()
    FN = ((predlist == 0) & (tarlist == 1)).sum()
    FP = ((predlist == 1) & (tarlist == 0)).sum()
    print(TP,TN,FN,FP)
    if TP+FP==0:
        p=0.5
    else:
        p = TP / (TP + FP)
    r = TP / (TP + FN)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    AUC = roc_auc_score(tarlist,scorelist)
    return TP,TN,FN,FP,p,r,F1,acc,AUC

### The Image Augmentation and DataLoader ###

In [7]:
DPATH = '/projectnb2/dl523/projects/COVIDCT2/dataset'
label_map = {"no IDC": 0, "IDC": 1}
torch.cuda.empty_cache()
#define image augmentation for training set and val&test set

train_transformer = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.8072870370812287, 0.6357799672097822, 0.7357258879210914],
                             std = [0.14257688600883628, 0.21227747141835426, 0.15241009580979106])
])

val_transformer = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(36),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.8072870370812287, 0.6357799672097822, 0.7357258879210914],
                        std = [0.14257688600883628, 0.21227747141835426, 0.15241009580979106])
])

class BreastCancerDataset(Dataset):
    def __init__(self, root_dir, npy_IDC, npy_NonIDC, transform=None, target_transform=None):
        """
        Args:
            root_dir (string): Path to all data
            npy_path (string): Path to the txt file with annotations.
            transform: image augmentation 
        """
        self.root_dir = root_dir
        self.npy_path = [npy_IDC, npy_NonIDC]
        X_IDC = np.load(os.path.join(self.root_dir, self.npy_path[0]))
        X_nonIDC = np.load(os.path.join(self.root_dir, self.npy_path[1]))
        self.X = np.concatenate((X_IDC, X_nonIDC))
        
        Y_IDC = np.ones(X_IDC.shape[0],dtype=np.int64)
        Y_nonIDC = np.zeros(X_nonIDC.shape[0],dtype=np.int64)
        self.Y = np.concatenate((Y_IDC, Y_nonIDC))
        
        assert np.issubdtype(self.Y.dtype, np.integer)
        assert self.Y.ndim == 1
        assert np.issubdtype(self.X.dtype, np.uint8)
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self,index):
        
        ## get image & label
        image, label = self.X[index], self.Y[index]
        
        ## transform 
        if self.transform is not None:
            image = self.transform(image)
            
        if self.target_transform is not None:
            label = self.target_transform(label)
        sample = {'img': image,
                  'label': label}
        return sample    




    
if __name__ == '__main__':
    npy_IDC_train = 'IDC/train_IDC.npy'

    npy_NonIDC_train = 'IDC/train_nonIDC.npy'


    trainset = BreastCancerDataset(DPATH, npy_IDC_train, npy_NonIDC_train,
                               transform=train_transformer)

    train_loader = DataLoader(trainset, batch_size=64, drop_last=True,shuffle=True, num_workers=16)
    npy_IDC_val = 'IDC/val_IDC.npy'

    npy_NonIDC_val = 'IDC/val_nonIDC.npy'


    valset = BreastCancerDataset(DPATH, npy_IDC_val, npy_NonIDC_val,
                               transform=val_transformer)

    val_loader = DataLoader(valset, batch_size=64, drop_last=False,shuffle=True, num_workers=16)
    npy_IDC_test = 'IDC/test_IDC.npy'

    npy_NonIDC_test = 'IDC/test_nonIDC.npy'


    testset = BreastCancerDataset(DPATH, npy_IDC_test, npy_NonIDC_test,
                               transform=val_transformer)

    test_loader = DataLoader(testset, batch_size=64, drop_last=False,shuffle=True, num_workers=16)
    
    print(trainset.__len__())
    print(valset.__len__())
    print(testset.__len__())
    

4437
555
558


In [8]:
testset.__getitem__(1)

{'img': tensor([[[ 0.8566,  0.9116,  0.8566,  ...,  0.2514,  0.3065,  0.4715],
          [ 0.9666,  0.9666,  0.9666,  ...,  0.1964,  0.2239,  0.3615],
          [ 0.9666,  0.9666,  0.8566,  ..., -0.3537, -0.1611, -0.1061],
          ...,
          [ 0.8841,  0.9391,  0.9666,  ...,  0.3890,  0.2789,  0.3065],
          [ 0.9391,  0.9391,  0.9116,  ...,  0.1689,  0.0314,  0.2239],
          [ 0.9391,  0.9116,  0.6640,  ..., -0.0786, -0.1336, -0.0236]],
 
         [[ 1.2170,  1.2355,  1.1800,  ..., -0.3533, -0.2794, -0.1870],
          [ 1.4017,  1.4202,  1.4017,  ..., -0.4641, -0.4272, -0.2055],
          [ 1.4202,  1.4017,  1.3094,  ..., -0.7412, -0.6304, -0.3533],
          ...,
          [ 1.3278,  1.3832,  1.3832,  ..., -0.7043, -0.7412, -0.6489],
          [ 1.3648,  1.3648,  1.3648,  ..., -0.6119, -0.5011, -0.2240],
          [ 1.3463,  1.3648,  1.0692,  ..., -0.6304, -0.3348, -0.2424]],
 
         [[ 1.1422,  1.1679,  1.1164,  ..., -0.3759, -0.2730, -0.2215],
          [ 1.3480,  

### Train, Validation and Test Function ###

In [9]:
def train(optimizer, epoch):
    
    model.train()
    
    train_loss = 0
    train_correct = 0
    total_num = 0 
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        data, target = batch_samples['img'].to(device), batch_samples['label'].to(device) 
        optimizer.zero_grad()
        output = model(data)
        criteria = nn.CrossEntropyLoss()
        loss = criteria(output, target.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_num += target.size(0)
        train_loss += loss.item()*target.size(0)                
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long().view_as(pred)).sum().item()
    
        if batch_index % 3 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()))
    return train_loss/total_num      

In [10]:
def val(epoch):
    
    model.eval()
    val_loss = 0
    total_num = 0 
    correct = 0
    results = []
   
    criteria = nn.CrossEntropyLoss()
    with torch.no_grad():
        
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            output = model(data)
            
            total_num += target.size(0)
            val_loss += criteria(output, target.long()).item()*target.size(0)
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
    return val_loss/total_num, targetlist, scorelist, predlist
    

In [11]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    total_num = 0 
    correct = 0
    results = []
    
    criteria = nn.CrossEntropyLoss()
    with torch.no_grad():
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        for batch_index, batch_samples in enumerate(test_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            output = model(data)
            test_loss += criteria(output, target.long())
            
            total_num += target.size(0)
            test_loss += criteria(output, target.long()).item()/target.size(0)
            
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
            
    return test_loss/total_num, targetlist, scorelist, predlist

### ResNet18 Backbone ### 

**SplitBatchNorm**: simulate the multiple gpu parallel computing in only one GPU. 

In [12]:
#modified ResNet18 model 
class SplitBatchNorm(nn.BatchNorm2d):
    def __init__(self, num_features, num_splits, **kw):
        super().__init__(num_features, **kw)
        self.num_splits = num_splits
        
    def forward(self, input):
        N, C, H, W = input.shape
        if self.training or not self.track_running_stats:
            running_mean_split = self.running_mean.repeat(self.num_splits)
            running_var_split = self.running_var.repeat(self.num_splits)
            outcome = nn.functional.batch_norm(
                input.view(-1, C * self.num_splits, H, W), running_mean_split, running_var_split, 
                self.weight.repeat(self.num_splits), self.bias.repeat(self.num_splits),
                True, self.momentum, self.eps).view(N, C, H, W)
            self.running_mean.data.copy_(running_mean_split.view(self.num_splits, C).mean(dim=0))
            self.running_var.data.copy_(running_var_split.view(self.num_splits, C).mean(dim=0))
            return outcome
        else:
            return nn.functional.batch_norm(
                input, self.running_mean, self.running_var, 
                self.weight, self.bias, False, self.momentum, self.eps)


class ModelBase(nn.Module):
    """
    Common CIFAR ResNet recipe.
    Comparing with ImageNet ResNet recipe, it:
    (i) replaces conv1 with kernel=3, str=1
    (ii) removes pool1
    """
    def __init__(self, feature_dim=128, arch=None, bn_splits=16):
        super(ModelBase, self).__init__()

        # use split batchnorm
        norm_layer = partial(SplitBatchNorm, num_splits=bn_splits) if bn_splits > 1 else nn.BatchNorm2d
        resnet_arch = getattr(resnet, arch)
        net = resnet_arch(num_classes=feature_dim, norm_layer=norm_layer)

        self.net = []
        for name, module in net.named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if isinstance(module, nn.MaxPool2d):
                continue
            if isinstance(module, nn.Linear):
                self.net.append(nn.Flatten(1))
            self.net.append(module)

        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        x = self.net(x)
        # note: not normalized here
        return x

In [20]:
model = ModelBase(arch='resnet18', bn_splits=8)
model.to(device)

#checkpoint = torch.load(args.model_load)

#for i in model.state_dict():
#    if 'encoder_q.'+i in checkpoint['state_dict']:
#        model.state_dict()[i].copy_(checkpoint['state_dict']['encoder_q.'+i].data)

    


ModelBase(
  (net): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel

### Load The MoCo Pretrained Encoder  and Start Final Training for the Classifier ###

In [ ]:
# train
args.resume = False
args.start_epoch = 1

votenum = 10

train_loss = np.zeros(1)
val_loss = np.zeros(1)
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

optimizer = optim.Adam(model.parameters(), lr=0.0001)

#for resuming 
if args.resume == True:
    results = pd.read_csv(args.results_dir + '/train_val_log.csv',index_col=0)
    checkpoint = torch.load(args.results_dir + '/model_last.pth')
    model.load_state_dict(checkpoint['state_dict'])
else: 
    results = {'epoch':[],'train_loss':[],'val_loss':[],'val_accurary': [],'val_F1': [],'val_AUC': []}


if not os.path.exists(args.results_dir):
    os.mkdir(args.results_dir)

for epoch in range(args.start_epoch, args.total_epoch+1):
    trainloss = train(optimizer, epoch)
    
    valloss,targetlist, scorelist, predlist = val(epoch)
    
    train_loss += trainloss
    val_loss += valloss 
    vote_pred += predlist 
    vote_score += scorelist 
    
    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        TP,TN,FN,FP,p,r,F1,acc,AUC = metric(vote_pred,targetlist,vote_score)
        results['epoch'].append(epoch)
        results['train_loss'].append((train_loss/votenum).item())
        results['val_loss'].append((val_loss/votenum).item())
        results['val_accurary'].append(acc)
        results['val_F1'].append(F1)
        results['val_AUC'].append(AUC)
        data_frame = pd.DataFrame(data=results)
        data_frame.to_csv(args.results_dir + '/train_val_log.csv')
        
        #save the most current model 
        torch.save({'epoch': epoch, 'state_dict': model.state_dict(),
                    'optimizer' : optimizer.state_dict(),}, args.results_dir + '/model_last.pth')
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, 10 epoch avg: train loss: {:.4f}, val loss: {:.4f}, acc: {:.4f}, F1: {:.4f}, AUC: {:.4f}'.format(
                epoch,(train_loss/votenum).item(),(val_loss/votenum).item(),acc,F1,AUC))
        
        train_loss = np.zeros(1)
        val_loss = np.zeros(1)
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())



# Final Results #

In [22]:
# test
model = ModelBase(arch='resnet18', bn_splits=8)
model.to(device)

checkpoint1 = torch.load('./IDC-cache/cache-train-Cifar-randomini/model_last.pth')
checkpoint2 = torch.load('./IDC-cache/cache-train-Cifar-IDC/model_last.pth')
#checkpoint3 = torch.load('Covid-cache/cache-train-Cifar-Covid/model_last.pth')
#checkpoint4 = torch.load('./cache-train-Cifar-Luna-Covid/model_last.pth')


### 1. Resnet18 Random Initialization (without Moco Pretraining) ### 
Final best performance:  acc, F1, AUC score: 0.810, 0.811, 0.901

In [23]:
#Resnet18 Random Initialization 
args.batch_size = 1

model.load_state_dict(checkpoint1['state_dict'])
epoch = 1

vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())


testloss,targetlist, scorelist, predlist = test(epoch)
#print('target',targetlist)
#print('score',scorelist)
#print('predict',predlist)
vote_pred = vote_pred + predlist 
vote_score = vote_score + scorelist 

TP,TN,FN,FP,p,r,F1,acc,AUC = metric(vote_pred,targetlist,vote_score)
print(acc,F1,AUC)


227 225 55 51
0.8100358422939068 0.8107142857142858 0.9007220680439922


### 2. two rounds of MoCo pretraining on Cifar-10 and IDC unlabeled Dataset  ### 
Final best performance:  acc, F1, AUC score: 0.821, 0.815, 0.907

In [24]:
#moco: Resnet18 pretrained on Cifar and IDC dataset
args.batch_size = 1

model.load_state_dict(checkpoint2['state_dict'])
epoch = 1

vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())


testloss,targetlist, scorelist, predlist = test(epoch)
#print('target',targetlist)
#print('score',scorelist)
#print('predict',predlist)
vote_pred = vote_pred + predlist 
vote_score = vote_score + scorelist 

TP,TN,FN,FP,p,r,F1,acc,AUC = metric(vote_pred,targetlist,vote_score)
print(acc,F1,AUC)



221 237 61 39
0.8207885304659498 0.8154981549815498 0.9069405899886935


### Partial Code Refer to: ### 
[link] (https://github.com/UCSD-AI4H/COVID-CT/blob/master/baseline%20methods/Self-Trans/CT-predict-pretrain.ipynb)


@article{zhao2020COVID-CT-Dataset,
  title={COVID-CT-Dataset: a CT scan dataset about COVID-19},
  author={Zhao, Jinyu and Zhang, Yichen and He, Xuehai and Xie, Pengtao},
  journal={arXiv preprint arXiv:2003.13865}, 
  year={2020}
}